# 🗿 Chapter A2: Model Loading (OBJ & GLTF)

Manually typing vertex data for complex models is impossible.

- A simple chair = 2,000 triangles
- A character = 10,000+ triangles

We need to **load models from files** created in Blender, Maya, or other 3D software.

## 1. The OBJ File Format

**Wavefront OBJ** is the simplest 3D format. It's **plain text**!

```
# Vertices
v 0.0 0.0 0.0
v 1.0 0.0 0.0
v 0.0 1.0 0.0

# Texture coordinates
vt 0.0 0.0
vt 1.0 0.0
vt 0.5 1.0

# Faces (v/vt/vn format: vertex/texture/normal)
f 1/1 2/2 3/3
```

- `v` = vertex position
- `vt` = texture coordinate (UV)
- `vn` = normal vector (for lighting)
- `f` = face (triangle)

## 2. Using PyWavefront

Don't parse OBJ files yourself! Use a library.

In [ ]:
# Install the library
!pip install pywavefront

In [ ]:
import pywavefront
import numpy as np

# Load the OBJ file
scene = pywavefront.Wavefront('models/character.obj', collect_faces=True)

# Get vertex data
vertices = np.array(scene.vertices, dtype='f4')

print(f"Loaded {len(vertices) // 8} vertices")  # 8 floats per vertex (X,Y,Z, NX,NY,NZ, U,V)

## 3. Creating VBO from Loaded Model

Convert the loaded data into a ModernGL buffer.

In [ ]:
import moderngl

# Assume 'ctx' is your ModernGL context
vbo = ctx.buffer(vertices.tobytes())

# Create VAO
# Format: position(3f) normal(3f) uv(2f)
vao = ctx.vertex_array(
    prog,
    [(vbo, '3f 3f 2f', 'in_vert', 'in_normal', 'in_uv')]
)

## 4. Alternative: Using Trimesh

`trimesh` is more powerful and supports more formats.

In [ ]:
# Install trimesh
!pip install trimesh

In [ ]:
import trimesh

# Load model
mesh = trimesh.load('models/robot.obj')

# Extract vertices and faces
vertices = mesh.vertices.astype('f4')  # (N, 3) array
faces = mesh.faces.astype('i4')  # (M, 3) array

# If model has UVs
if hasattr(mesh.visual, 'uv'):
    uvs = mesh.visual.uv.astype('f4')

print(f"Vertices: {len(vertices)}, Faces: {len(faces)}")

## 5. Indexed Rendering (Vertex Reuse)

A cube has **8 vertices** but **12 triangles** (36 vertex references).

Without indices: 36 vertices
With indices: 8 vertices + 36 indices = **78% less data**

In [ ]:
# Create index buffer
indices = mesh.faces.flatten().astype('i4')
ibo = ctx.buffer(indices.tobytes())

# Create VAO with index buffer
vao = ctx.vertex_array(
    prog,
    [(vbo, '3f 3f 2f', 'in_vert', 'in_normal', 'in_uv')],
    ibo
)

# Render with indices
vao.render()

## 6. GLTF Format (Modern Standard)

**GLTF** is the "JPEG of 3D". It's used by:
- Sketchfab
- Unity
- Unreal Engine
- Web3D (Three.js)

It supports:
- Animations
- Multiple materials
- PBR textures
- Skeletal rigs

In [ ]:
# Load GLTF with trimesh
scene = trimesh.load('models/character.gltf')

# GLTF files can contain multiple meshes
for name, geometry in scene.geometry.items():
    print(f"Mesh: {name}, Vertices: {len(geometry.vertices)}")

## 7. Loading Textures from Model

GLTF files include material/texture references.

In [ ]:
from PIL import Image

# Get material
material = list(scene.geometry.values())[0].visual.material

# Load diffuse texture
if hasattr(material, 'baseColorTexture'):
    texture_image = material.baseColorTexture
    
    # Convert to OpenGL texture
    img_data = texture_image.tobytes()
    texture = ctx.texture(texture_image.size, 3, img_data)
    texture.build_mipmaps()

## 🛠️ Challenge: The Animated Character

Download a free character model from:
- [Poly Pizza](https://poly.pizza) (CC0 license)
- [Sketchfab](https://sketchfab.com/tags/free) (filter by free downloads)

1. Load the model using `trimesh` or `pywavefront`
2. Extract vertex data and create a VAO
3. Render the character in your 3D world
4. Make it rotate slowly on the Y axis

**Bonus:** Load multiple models and create a small 3D scene (trees, rocks, buildings)

If you can walk around a scene with imported models, you're ready for real game development. 🎮